# Building a Convolutional Neural Net with Keras & Tensorflow

This tutorial demonstrates how to build a deep Convolutional Neural Network for image classification with Keras on a Tensorflow backend using the 
[CIFAR-10 dataset](http://www.cs.toronto.edu/~kriz/cifar.html) -  
It will also demonstrate how easily Tensorboard can be utilized with Keras. This notebook was created with the help of [an example implementation](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py) from the Keras-team.

This notebook is intended to be run on [Google Cloud Datalab](https://cloud.google.com/datalab/)

Datalab will have the required libraries installed by default for this code to work. If you choose to run this code outside of Datalab you may run in to version and dependency issues which you will need to resolve.

In [ ]:
from __future__ import division
from __future__ import print_function

!pip3 install keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
"""Loads CIFAR10 dataset.
  Returns:
      Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
      Source: https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/python/keras/datasets/cifar10.py
  """

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Defining class names and how many there are
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = 10

In [ ]:
# Checking dataset and printing figures of 10 random images from each class
fig = plt.figure(figsize=(8,3))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = x_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = np.transpose(features_idx[img_num,::],(0,1,2))
    ax.set_title(class_names[i])
    plt.imshow(im)
plt.show()

In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Create the Model

Start building the convnet using Keras' [Sequential Model API](https://keras.io/models/sequential/) by 'stacking up' the layers.

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Setting up Tensorboard

In [ ]:
# Instatiating Keras' Tensorboard class with all the parts we want to have included; this will be added into .fit-methods callback for training
tensorboard = TensorBoard(
    log_dir='./logs',#{}.format('cnn'),
    write_graph=True,
    write_grads=True,
    histogram_freq=1,
    write_images=True,
)

The following code will instatiate a locally hosted TensorBoard and provide a link to access it.

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('./logs')

The following code will stop Tensorboard - RUN ONLY IF YOU WANT TO CLOSE IT!!

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)

# Compile and Train the Model

In [ ]:
# Let's compile the model using Adam Optimizer incl. a learning rate + decay
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001,decay=1e-6),
              metrics=['accuracy'])

# Let's normalize the features
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
# Let's train the model
model.fit(x_train, y_train,
              batch_size=128,
              epochs=5,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=[tensorboard])

# Test the Model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)

print('test loss: {:.4f}'.format(score[0]))
print(' test acc: {:.4f}'.format(score[1]))